<font color=red size=6>**Orbitales Moleculares**
    
<font color=red size=5>**Enrique Mejía Ospino**
    
<font color=red size=5>**Química Cuántica Avanzada**
    
<font color=red size=5>**Universidad Industrial de Santander**

In [1]:
import numpy
from pyscf import gto, scf, lo, tools
from functools import reduce

benzene = [[ 'C'  , ( 4.673795 ,   6.280948 , 0.00  ) ],
           [ 'C'  , ( 5.901190 ,   5.572311 , 0.00  ) ],
           [ 'C'  , ( 5.901190 ,   4.155037 , 0.00  ) ],
           [ 'C'  , ( 4.673795 ,   3.446400 , 0.00  ) ],
           [ 'C'  , ( 3.446400 ,   4.155037 , 0.00  ) ],
           [ 'C'  , ( 3.446400 ,   5.572311 , 0.00  ) ],
           [ 'H'  , ( 4.673795 ,   7.376888 , 0.00  ) ],
           [ 'H'  , ( 6.850301 ,   6.120281 , 0.00  ) ],
           [ 'H'  , ( 6.850301 ,   3.607068 , 0.00  ) ],
           [ 'H'  , ( 4.673795 ,   2.350461 , 0.00  ) ],
           [ 'H'  , ( 2.497289 ,   3.607068 , 0.00  ) ],
           [ 'H'  , ( 2.497289 ,   6.120281 , 0.00  ) ]]

mol = gto.M(atom=benzene,
            basis='ccpvdz')
mf = scf.RHF(mol).run()

mo_occ = mf.mo_coeff[:,mf.mo_occ>0]
a = lo.iao.iao(mol, mo_occ)

# Orthogonalize IAO
a = lo.vec_lowdin(a, mf.get_ovlp())

#
# Method 1, using Knizia's alogrithm to localize IAO orbitals
#
'''
Generate IBOS from orthogonal IAOs
'''

ibo = lo.ibo.ibo(mol, mo_occ, iaos=a)

'''
Print the IBOS into Gausian Cube files
'''

for i in range(ibo.shape[1]):
    tools.cubegen.orbital(mol, './Data/cube/benzene_ibo1_{:02d}.cube'.format(i+1), ibo[:,i])

'''
Population Analysis with IAOS
'''
# transform mo_occ to IAO representation. Note the AO dimension is reduced
mo_occ = reduce(numpy.dot, (a.T, mf.get_ovlp(), mo_occ))

#constructs the density matrix in the new representation
dm = numpy.dot(mo_occ, mo_occ.T) * 2

#mullikan population analysis based on IAOs
iao_mol = mol.copy()
iao_mol.build(False, False, basis='minao')
mf.mulliken_pop(iao_mol, dm, s=numpy.eye(iao_mol.nao_nr()))

#
# Method 2, using the modified Pipek-Mezey localization module.
# Orthogonalization for IAOs is not required.
#
mo_occ = mf.mo_coeff[:,mf.mo_occ>0]
iaos = lo.iao.iao(mol, mo_occ)
ibo = lo.ibo.ibo(mol, mo_occ, locmethod='PM', iaos=iaos).kernel()
for i in range(ibo.shape[1]):
    tools.cubegen.orbital(mol, './Data/cube/benzene_ibo2_{:02d}.cube'.format(i+1), ibo[:,i])

/home/emejia/psi4conda/envs/psi4/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


converged SCF energy = -230.716652171385
 Iterative localization: IB/P4/2x2, 10 iter; Final gradient 5.91e-09
 ** Mulliken pop  **
pop of  0 C 1s        1.98981
pop of  0 C 2s        0.98949
pop of  0 C 2px       1.03857
pop of  0 C 2py       1.11832
pop of  0 C 2pz       1.00000
pop of  1 C 1s        1.98981
pop of  1 C 2s        0.98949
pop of  1 C 2px       1.09838
pop of  1 C 2py       1.05851
pop of  1 C 2pz       1.00000
pop of  2 C 1s        1.98981
pop of  2 C 2s        0.98949
pop of  2 C 2px       1.09838
pop of  2 C 2py       1.05851
pop of  2 C 2pz       1.00000
pop of  3 C 1s        1.98981
pop of  3 C 2s        0.98949
pop of  3 C 2px       1.03857
pop of  3 C 2py       1.11832
pop of  3 C 2pz       1.00000
pop of  4 C 1s        1.98981
pop of  4 C 2s        0.98949
pop of  4 C 2px       1.09838
pop of  4 C 2py       1.05851
pop of  4 C 2pz       1.00000
pop of  5 C 1s        1.98981
pop of  5 C 2s        0.98949
pop of  5 C 2px       1.09838
pop of  5 C 2py       1.05851

Overwritten attributes  atomic_pops  of <class 'pyscf.lo.pipek.PipekMezey'>


In [5]:
from pyscf import lo
from pyscf.tools import molden
mol = gto.M(
    atom = '''
  C  3.2883    3.3891    0.2345
  C  1.9047    3.5333    0.2237
  C  3.8560    2.1213    0.1612
  C  1.0888    2.4099    0.1396
  C  3.0401    0.9977    0.0771
  C  1.6565    1.1421    0.0663
  H  3.9303    4.2734    0.3007
  H  1.4582    4.5312    0.2815
  H  4.9448    2.0077    0.1699
  H  0.0000    2.5234    0.1311
  H  3.4870    0.0000    0.0197
  H  1.0145    0.2578    0.0000
           ''',
    basis = 'cc-pvdz',
    symmetry = 1)

mf = scf.RHF(mol)
mf.kernel()

#
# First method is to explicit call the functions provided by molden.py
#
with open('./Data/C6H6mo.molden', 'w') as f1:
    molden.header(mol, f1)
    molden.orbital_coeff(mol, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)


converged SCF energy = -230.721654045954


In [10]:
import numpy
from pyscf import gto, scf
from pyscf import lo
from pyscf.tools import molden

mol = gto.M(
    atom = '''
C    0.000000000000     1.398696930758     0.000000000000
C    0.000000000000    -1.398696930758     0.000000000000
C    1.211265339156     0.699329968382     0.000000000000
C    1.211265339156    -0.699329968382     0.000000000000
C   -1.211265339156     0.699329968382     0.000000000000
C   -1.211265339156    -0.699329968382     0.000000000000
H    0.000000000000     2.491406946734     0.000000000000
H    0.000000000000    -2.491406946734     0.000000000000
H    2.157597486829     1.245660462400     0.000000000000
H    2.157597486829    -1.245660462400     0.000000000000
H   -2.157597486829     1.245660462400     0.000000000000
H   -2.157597486829    -1.245660462400     0.000000000000''',
    basis = '6-31g')
mf = scf.RHF(mol)
mf.chkfile = './Data/benzene-631g.chk'
mf.kernel()

pz_idx = numpy.array([17,20,21,22,23,30,36,41,42,47,48,49])-1
loc_orb = lo.Boys(mol, mf.mo_coeff[:,pz_idx]).kernel()
molden.from_mo(mol, './Data/benzene-631g-boys.molden', loc_orb)

converged SCF energy = -230.622212089114


In [8]:
from pyscf import gto, scf
from pyscf.tools import cubegen

mol = gto.M(atom='''
            O 0.0000000, 0.000000, 0.00000000
            H 0.761561 , 0.478993, 0.00000000
            H -0.761561, 0.478993, 0.00000000''', basis='6-31g*')
mf = scf.RHF(mol).run()

# electron density
cubegen.density(mol, './Data/cube/h2o_den.cube', mf.make_rdm1())

# molecular electrostatic potential
cubegen.mep(mol, './Data/cube/h2o_pot.cube', mf.make_rdm1())

# 1st MO
cubegen.orbital(mol, './Data/cube/h2o_mo1.cube', mf.mo_coeff[:,0])


converged SCF energy = -76.0019261895009


array([[[1.79667191e-06, 1.92996194e-06, 2.06923789e-06, ...,
         2.06923789e-06, 1.92996194e-06, 1.79667191e-06],
        [2.00043883e-06, 2.14868745e-06, 2.30357385e-06, ...,
         2.30357385e-06, 2.14868745e-06, 2.00043883e-06],
        [2.22175713e-06, 2.38622693e-06, 2.55803810e-06, ...,
         2.55803810e-06, 2.38622693e-06, 2.22175713e-06],
        ...,
        [5.87903518e-06, 6.31941859e-06, 6.78011205e-06, ...,
         6.78011205e-06, 6.31941859e-06, 5.87903518e-06],
        [5.42521484e-06, 5.83176540e-06, 6.25708580e-06, ...,
         6.25708580e-06, 5.83176540e-06, 5.42521484e-06],
        [4.99392093e-06, 5.36829033e-06, 5.75996111e-06, ...,
         5.75996111e-06, 5.36829033e-06, 4.99392093e-06]],

       [[1.99570771e-06, 2.14338948e-06, 2.29765641e-06, ...,
         2.29765641e-06, 2.14338948e-06, 1.99570771e-06],
        [2.22148959e-06, 2.38567190e-06, 2.55714899e-06, ...,
         2.55714899e-06, 2.38567190e-06, 2.22148959e-06],
        [2.46662787e-06, 

**<font color=blue> En este ejemplo se realiza el cálculos Orbitales Moleculares (OM) utlizando el modulo *PSI4* y que son guardados en un Gaussian .fchk.** 

In [2]:
# Inicialización Psi4 =================================================================
import psi4

psi4.core.clean()
psi4.core.clean_options()
psi4.set_memory('2000 MB')  # Can make this much larger on Seawulf, each compute node has more than 100 GB RAM
psi4.set_num_threads(4)    # Can make this much larger on Seawulf, each compute node can support 24 threads. 
                          # But it doesn't help much for small molecules...
psi4.core.set_output_file('./Data/OM1.dat', False) 
# this command sets psi4's output to a file. Comment this line out if you want to see the output on the terminal.


  Memory set to   1.863 GiB by Python driver.
  Threads set to 4 by Python driver.


In [4]:
# Inicialización de los cálculos ================================================

# En este ejemplo se carga la molécula de un archivo en formato .xyz
 
f = open('./molecules/Aniline.xyz')
molecula = psi4.geometry(f.read())
molecula.reset_point_group('c1') # Simetría!

metodo1 = 'scf/3-21G'  # Métodos de cálculos.
metodo2 = 'scf/6-311G*'

E0_1, wfn1 = psi4.energy(metodo1, molecule = molecula, return_wfn = True)  # cálculo de energía usando metodo1
E0_2, wfn2 = psi4.energy(metodo2, molecule = molecula, return_wfn = True)  # cálculo de energía usando metodo2

E0_3, wfn3 = psi4.gradient(metodo1, molecule = molecula, return_wfn = True)  # cálculo de gradiente usando metodo1
E0_4, wfn4 = psi4.gradient(metodo2, molecule = molecula, return_wfn = True)  # cálculo de gradiente usando metodo2

# Se guardan en archivos fchk con los diferentes bases para ver los OM con avogadro o gaussview
psi4.fchk(wfn1, './Data/OM_Aniline_HF_621g.fchk')
psi4.fchk(wfn2, './Data/OM_Aniline_HF_6311g.fchk')

# Se guardan en archivos fchk con los diferentes bases para ver las densidades electrónicas
psi4.fchk(wfn3, './Data/DE_Aniline_HF_621g.fchk')
psi4.fchk(wfn4, './Data/DE_Aniline_HF_6311g.fchk')